In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from models import AttentionResNet92, AttentionResNet56
from tensorflow.keras import optimizers
import time

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(featurewise_center=True,
        featurewise_std_normalization=True,
        rescale=1./255,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/data_2categ/dataset_augmented/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/data_2categ/dataset_augmented/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

Found 7871 images belonging to 2 classes.
Found 1966 images belonging to 2 classes.


In [3]:
NAME = "dimentia-residual-attention-CNN-{}".format(int(time.time()))

# build the model
model = AttentionResNet56(n_classes=1) # here n_classes represent neuron in final layer
model.summary()

AttributeError: 'int' object has no attribute 'value'

In [ ]:
# plot model graph
#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
# prepare usefull callbacks
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=7, min_lr=10e-7, epsilon=0.01, verbose=1)
#early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1)

In [ ]:
def lr_schedule(epoch):

    lr = 1e-4 #1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60: # 120
        lr *= 1e-2
    elif epoch > 30: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)), # (lr=lr_schedule(0)),
              metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
# fits the model on batches with real-time data augmentation

filepath="best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,
                    callbacks=callbacks

In [ ]:
model.evaluate_generator(generator=test_generator,
steps=STEP_SIZE_TEST)